In [1]:
import wfc
from wfc import *
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [8]:
from lasagna.imports import *
import lasagna.io
j = lasagna.config.j = start_client()
dr = ((0, 1),)*100
def show(*args, **kwargs):
    lasagna.io.show_IJ(*args, display_ranges=dr, **kwargs)

In [62]:
N = 3
FMX = 20
FMY = 20

a = (1, 0, 1)
b = (0, 1, 0)
c = (1, 1, 1)
d = (0, 0, 0)

T = rotate((c,b,b), 4)
X = rotate((a,b,a), 1)
I = rotate((c,b,c), 2)

initial = [(a,b,a), (b,a,b)]
# initial = T

filename = 'samples/Dungeon.png'
initial = initial_from_png(filename, N)
# exclude = [((0, 0, 1), (1, 1, 1), (1, 1, 1)),
#            ((0, 1, 1), (0, 1, 1), (1, 1, 1)),
#            ((1, 1, 1), (1, 1, 0), (1, 1, 0)),
#            ((1, 1, 1), (1, 1, 1), (1, 0, 0))]
# initial = [x for x in initial if x not in exclude]

model = SimpleTiledModel(N,FMX,FMY,initial, periodic=True)
print 'initial has %d unique tiles' % len(model.initial)
arr = []
np.random.seed(2)

initial has 52 unique tiles


In [66]:
model = reset(seed=1)
%lprun -f model.update model.complete(support=True)

In [59]:
arr = []
for seed in range(30):
    model = reset(seed=seed)
    try:
        model.complete(support=False)
        arr += [model.get_superposition()]
    except AssertionError:
        pass
show(arr, 'randos')

Cast float64 to float32


In [11]:
def reset(seed=0):
    np.random.seed(seed)
    wfc.ticker[0] = seed
    return SimpleTiledModel(N,FMX,FMY,initial, periodic=True)

In [55]:
%lprun -f model.propagate_support do_it(support=True)

collapsed (0, 0)
collapsed (0, 1)
collapsed (0, 2)
collapsed (1, 1)
collapsed (2, 1)
collapsed (2, 0)
collapsed (3, 0)
collapsed (2, 3)
collapsed (0, 4)
collapsed (1, 5)
collapsed (0, 5)
collapsed (1, 6)
collapsed (4, 5)
collapsed (0, 7)
collapsed (0, 8)
collapsed (5, 7)
collapsed (1, 10)
collapsed (0, 10)
collapsed (1, 11)
collapsed (0, 11)
collapsed (2, 11)
collapsed (3, 10)
collapsed (1, 12)
collapsed (0, 12)
collapsed (1, 13)
collapsed (0, 13)
collapsed (2, 13)
collapsed (3, 12)
collapsed (4, 9)
collapsed (5, 9)
collapsed (3, 14)
collapsed (4, 14)
collapsed (0, 14)
collapsed (1, 15)
collapsed (0, 16)
collapsed (1, 16)
collapsed (2, 15)
collapsed (4, 15)
collapsed (0, 17)
collapsed (28, 15)
collapsed (29, 16)
collapsed (0, 18)
collapsed (1, 18)
collapsed (2, 18)
collapsed (29, 18)
collapsed (0, 20)
collapsed (4, 29)
collapsed (29, 19)
collapsed (1, 20)
collapsed (2, 21)
collapsed (0, 22)
collapsed (0, 28)
collapsed (0, 29)
collapsed (1, 22)
collapsed (1, 28)
collapsed (1, 29)
collap

In [80]:
print model.wave.sum()
model.update(support=True)
print model.wave.sum()

44100
collapsed (22, 24)
40908


In [107]:
arr = []
def do_it(support=False, record=0):
    global arr
    for i in range(30000):
        done = model.update(support=support)
        if done:
            break
        if record and (i % record) == 0:
            arr += [model.get_superposition()]
    print i
    return arr

In [61]:
show(model.get_superposition(), 'superposition')
# show(model.get_entropy(), 'entropy')
# show(model.wave.sum(axis=0), 'wavesum')
# show(model.initial, 'initial')

Cast float64 to float32


In [27]:
arr = np.array(arr)
skimage.external.tifffile.imsave('deleteme.tif', arr.astype(np.float32))

In [86]:
model.wave = np.random.choice([True, False], p=[0.9,0.1], size=model.wave.shape)
# model.wave[0,:,:] = True

In [89]:
%timeit -n1 -r1 model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 2808 points, touched 27036
1 loop, best of 1: 2.56 s per loop


In [99]:
%prun -s cumulative model.propagate_from(zip_where(model.wave.sum(axis=0)))

propagation modified 0 points, touched 5512
 

In [105]:
import pyperclip
show(eval(pyperclip.paste()))

Cast int64 to int16 without loss
